## IMPORT

In [1]:
import warnings
import numpy                    as np
from keras.models               import Sequential
from keras.layers               import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image  import ImageDataGenerator
from keras.preprocessing        import image


warnings.filterwarnings("ignore", category=DeprecationWarning)

#### CLASSE PARA CRIAR O MODELO

In [2]:
# Criação da classe junto com os métodos necessários, como fit, predict e save. 
# Além da preparação de imagens para ser usada no modelo.
class Model():
    def __init__(self):
        self.clf = Sequential()
        self.clf.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
        self.clf.add(BatchNormalization())
        self.clf.add(MaxPooling2D(pool_size = (2, 2)))

        self.clf.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
        self.clf.add(BatchNormalization())
        self.clf.add(MaxPooling2D(pool_size = (2, 2)))
        
        self.clf.add(Flatten())
        self.clf.add(Dense(units = 128, activation='relu'))
        self.clf.add(Dropout(0.2))
        self.clf.add(Dense(units = 128, activation='relu'))
        self.clf.add(Dropout(0.2))
        self.clf.add(Dense(units = 128, activation='relu'))
        self.clf.add(Dropout(0.2))
        self.clf.add(Dense(units = 1, activation = 'sigmoid'))
    
        self.clf.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    def data_prepare(self):
        training_generator = ImageDataGenerator(rescale = 1./255,
                                        rotation_range = 7,
                                        horizontal_flip = True,
                                        shear_range = 0.2,
                                        height_shift_range = 0.07,
                                        zoom_range = 0.2)

        test_generator = ImageDataGenerator(rescale = 1./255)
        
        base_training = training_generator.flow_from_directory('dataset/training_set', 
                                                          target_size = (64, 64),
                                                          batch_size = 2,
                                                          class_mode = 'binary')
        base_test = test_generator.flow_from_directory('dataset/test_set', target_size = (64, 64),
                                                          batch_size = 2,
                                               class_mode = 'binary')
        return (base_training, base_test)
        
    def to_train(self, base_training, base_test):
        self.clf.fit(base_training, epochs = 10, validation_data = base_test)
    
    def save(self, path):
        self.clf.save(r'{}'.format(path))
    
    def predict(self, path):
        img_test = image.load_img('{}'.format(path), target_size = (64, 64))
        img_test = image.img_to_array(img_test)
        img_test /= 255
        img_test = np.expand_dims(img_test, axis = 0)
        prev = self.clf.predict(img_test)
        if prev >= 0.5:
            return 'Cat'
        else:
            return 'Dog'

In [3]:
model = Model()
base_training, base_test = model.data_prepare()

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
model.to_train(base_training, base_test)

Epoch 1/10
4000/4000 [==============================] - 70s 18ms/step - loss: 0.7553 - accuracy: 0.5615 - val_loss: 0.6531 - val_accuracy: 0.6265
Epoch 2/10
4000/4000 [==============================] - 65s 16ms/step - loss: 0.6556 - accuracy: 0.6260 - val_loss: 0.6235 - val_accuracy: 0.6670
Epoch 3/10
4000/4000 [==============================] - 65s 16ms/step - loss: 0.6363 - accuracy: 0.6519 - val_loss: 0.5791 - val_accuracy: 0.7155
Epoch 4/10
4000/4000 [==============================] - 68s 17ms/step - loss: 0.6134 - accuracy: 0.6771 - val_loss: 0.6575 - val_accuracy: 0.6915
Epoch 5/10
4000/4000 [==============================] - 76s 19ms/step - loss: 0.5910 - accuracy: 0.6967 - val_loss: 0.5560 - val_accuracy: 0.7255
Epoch 6/10
4000/4000 [==============================] - 74s 19ms/step - loss: 0.5767 - accuracy: 0.7151 - val_loss: 0.5503 - val_accuracy: 0.7280
Epoch 7/10
4000/4000 [==============================] - 74s 18ms/step - loss: 0.5552 - accuracy: 0.7230 - val_loss: 0.5663 -

In [5]:
# teste para uma imagem. É passado o caminho da imagem
print(model.predict('dataset/test_set/gato/cat.4031.jpg'))

Cat


In [8]:
# salvando o modelo. É necessário inserir o caminho.
path = r'C:\Users\vinic\Desktop\estudo\programacao\python\projetos_ML\cats-and-dogs'
model.save(path)

INFO:tensorflow:Assets written to: C:\Users\vinic\Desktop\estudo\programacao\python\projetos_ML\cats-and-dogs\assets
